# Домашнее задание 3

1) [1 балл] Реализуйте чтение изображений, лежащих в /data/imgs/arrow (как в демо с калибровкой камеры с семинара). Вся обязательная часть задания выполняется с этими изображениями

1) [2 балла] Реализуйте нахождение расстояния от камеры до листка, учитывая, что это тот же листок, что был в первом демо

2) [3 балла] Реализуйте приведение изображения листка к "виду сверху"

3) [6 баллов] Реализуйте определение направления, в котором указывает стрелка. Можно использовать скелетонизацию из первого семинара по зрению: получить маску темной области (стрелки) внутри изображения листка, применить скелетонизацию, вычислить направление. Можно поступить иначе: после нахождения маски вычислить градиенты Собеля по x и y и проанализировать их направления. P.S. Можно использовать и полностью свой метод

4) [3 балла] Доп. задача 1: Примените написанный Вами код к видео https://drive.google.com/file/d/1AQhg2lL5FL4edlOcxY2SHOOZ-v9TgHBD/view?usp=sharing. Загрузите видео на свой гугл диск, а затем дополните имеющийся код монтированием диска и чтением в цикле кадров видео, приведёнными в конце семинара

5) [5 баллов] Доп. задача 2: определите, нормально ли видно листок в кадре. На некоторых кадрах его видно не полностью, один или больше уголков находятся за кадром. Это можно автоматически обнаруживать, к примеру используя размеры стрелки, и выводить на соответствующем кадре маркер - текст или цветную метку

In [1]:
!git clone https://github.com/geomags3/cvr_course.git

Cloning into 'cvr_course'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 177 (delta 6), reused 0 (delta 0), pack-reused 165
Receiving objects: 100% (177/177), 37.86 MiB | 16.94 MiB/s, done.
Resolving deltas: 100% (43/43), done.


In [9]:
from pathlib import Path
import numpy as np
import cv2
import math
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10, 10)

from ipywidgets import interact
import ipywidgets as widgets

In [8]:
# Path to the arrow images
arrows_folder = Path('/content/cvr_course/data/imgs/arrow')
# List of arrow images
arrow_files = list(arrows_folder.iterdir())


[PosixPath('/content/cvr_course/data/imgs/arrow/vlcsnap-2020-10-21-18h01m27s294.png'),
 PosixPath('/content/cvr_course/data/imgs/arrow/vlcsnap-2020-10-21-18h01m51s638.png'),
 PosixPath('/content/cvr_course/data/imgs/arrow/vlcsnap-2020-10-21-18h01m29s941.png'),
 PosixPath('/content/cvr_course/data/imgs/arrow/vlcsnap-2020-10-21-18h01m41s874.png'),
 PosixPath('/content/cvr_course/data/imgs/arrow/vlcsnap-2020-10-21-18h01m36s056.png'),
 PosixPath('/content/cvr_course/data/imgs/arrow/vlcsnap-2020-10-21-18h01m44s163.png'),
 PosixPath('/content/cvr_course/data/imgs/arrow/vlcsnap-2020-10-21-18h01m46s644.png'),
 PosixPath('/content/cvr_course/data/imgs/arrow/vlcsnap-2020-10-21-18h01m22s212.png'),
 PosixPath('/content/cvr_course/data/imgs/arrow/vlcsnap-2020-10-21-18h01m49s487.png'),
 PosixPath('/content/cvr_course/data/imgs/arrow/vlcsnap-2020-10-21-18h01m33s362.png')]

### Чтение изображений

In [14]:
@interact(file_idx=(0, len(arrow_files) - 1))
def show_files(file_idx=0):
  img = cv2.imread(str(arrow_files[file_idx]))
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

  plt.imshow(img)
  plt.axis('off')
  plt.show()

interactive(children=(IntSlider(value=0, description='file_idx', max=9), Output()), _dom_classes=('widget-inte…

### Определение расстояния до листка